To build a lookalike model that recommends similar customers based on profile and transaction history, we can use a similarity-based approach such as cosine similarity or Euclidean distance.

1. Import Libraries and Load Data


In [19]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [21]:
# Load data
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

2.Merge Data

In [23]:
# Merge transactions with products and customers
merged_data = transactions.merge(products, on='ProductID').merge(customers, on='CustomerID')

# Aggregate data to create customer profiles
customer_profiles = merged_data.groupby('CustomerID').agg({
    'Category': lambda x: x.value_counts().to_dict(),  # Frequency of product categories
    'TotalValue': 'sum',                              # Total spend
    'TransactionID': 'count',                         # Frequency of purchases
}).reset_index()

# Rename columns for clarity
customer_profiles.rename(columns={'TransactionID': 'PurchaseFrequency', 'TotalValue': 'TotalSpend'}, inplace=True)

In [25]:
merged_data

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00761,C0199,P022,2024-10-01 05:57:09,4,550.16,137.54,HomeSense Wall Art,Home Decor,137.54,Andrea Jenkins,Europe,2022-12-03
2,T00626,C0199,P079,2024-08-17 12:06:08,2,834.74,417.37,ActiveWear Rug,Home Decor,417.37,Andrea Jenkins,Europe,2022-12-03
3,T00963,C0199,P008,2024-10-26 00:01:58,2,293.70,146.85,BookWorld Bluetooth Speaker,Electronics,146.85,Andrea Jenkins,Europe,2022-12-03
4,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00774,C0095,P056,2024-01-07 14:19:49,2,32.16,16.08,SoundWave Smartwatch,Electronics,16.08,William Walker,South America,2023-03-04
996,T00823,C0095,P079,2024-09-30 10:45:06,3,1252.11,417.37,ActiveWear Rug,Home Decor,417.37,William Walker,South America,2023-03-04
997,T00369,C0151,P082,2024-12-24 11:40:24,4,223.96,55.99,ComfortLiving Rug,Home Decor,55.99,Amber Gonzalez,South America,2024-11-22
998,T00809,C0078,P075,2024-12-09 11:44:44,2,995.52,497.76,TechPro Textbook,Books,497.76,Julia Palmer,Asia,2024-11-13


3.Encode Features

In [44]:
# One-hot encode product categories
category_df = merged_data.pivot_table(index='CustomerID', columns='Category', aggfunc='size', fill_value=0)
customer_profiles = customer_profiles.merge(category_df, on='CustomerID', how='left')

# Normalize numerical features
scaler = MinMaxScaler()
customer_profiles[['TotalSpend', 'PurchaseFrequency']] = scaler.fit_transform(customer_profiles[['TotalSpend', 'PurchaseFrequency']])

In [45]:
category_df

Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,1,0,3,1
C0002,0,2,0,2
C0003,0,1,1,2
C0004,3,0,2,3
C0005,0,0,2,1
...,...,...,...,...
C0196,1,1,0,2
C0197,0,0,2,1
C0198,0,1,1,0


In [29]:
customer_profiles

,CustomerID,Category,TotalSpend,PurchaseFrequency,Books,Clothing,Electronics,Home Decor
0,C0001,"{'Electronics': 3, 'Books': 1, 'Home Decor': 1}",0.308942,0.4,1,0,3,1
1,C0002,"{'Home Decor': 2, 'Clothing': 2}",0.168095,0.3,0,2,0,2
2,C0003,"{'Home Decor': 2, 'Clothing': 1, 'Electronics'...",0.249541,0.3,0,1,1,2
3,C0004,"{'Books': 3, 'Home Decor': 3, 'Electronics': 2}",0.497806,0.7,3,0,2,3
4,C0005,"{'Electronics': 2, 'Home Decor': 1}",0.184287,0.2,0,0,2,1
...,...,...,...,...,...,...,...,...
194,C0196,"{'Home Decor': 2, 'Books': 1, 'Clothing': 1}",0.462684,0.3,1,1,0,2
195,C0197,"{'Electronics': 2, 'Home Decor': 1}",0.174318,0.2,0,0,2,1
196,C0198,"{'Electronics': 1, 'Clothing': 1}",0.080203,0.1,0,1,1,0
197,C0199,"{'Electronics': 2, 'Home Decor': 2}",0.179098,0.3,0,0,2,2


4. Compute Cosine Similarity


In [62]:
# Compute cosine similarity between customers
customer_features = customer_profiles.drop(['CustomerID'], axis=1).select_dtypes(include=['number'])
similarity_matrix = cosine_similarity(customer_features)
# Convert similarity matrix into a DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles['CustomerID'], columns=customer_profiles['CustomerID'])
similarity_df

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.220272,0.620054,0.774347,0.944141,0.378079,0.944190,0.773232,0.408210,0.102455,...,0.755057,0.863082,0.306676,0.838171,0.489948,0.378474,0.944103,0.641470,0.853934,0.576059
C0002,0.220272,1.000000,0.866848,0.457810,0.321047,0.580577,0.321381,0.776343,0.634420,0.672901,...,0.008086,0.294821,0.005993,0.592383,0.944906,0.864028,0.320972,0.502546,0.503723,0.801668
C0003,0.620054,0.866848,1.000000,0.700539,0.732617,0.507284,0.732992,0.968756,0.550202,0.391994,...,0.234586,0.505561,0.008040,0.795332,0.982185,0.834602,0.732522,0.579706,0.866941,0.774558
C0004,0.774347,0.457810,0.700539,1.000000,0.670379,0.786407,0.670867,0.820038,0.197090,0.209579,...,0.771081,0.614129,0.641826,0.946957,0.632351,0.785971,0.670264,0.306603,0.755840,0.651089
C0005,0.944141,0.321047,0.732617,0.670379,1.000000,0.191984,0.999869,0.817225,0.403002,0.006214,...,0.500129,0.732404,0.006162,0.745991,0.601301,0.372330,0.999995,0.634307,0.948907,0.512361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0196,0.378474,0.864028,0.834602,0.785971,0.372330,0.836844,0.374069,0.819794,0.368379,0.518793,...,0.347717,0.341970,0.413079,0.794631,0.873354,1.000000,0.371982,0.294048,0.580282,0.776846
C0197,0.944103,0.320972,0.732522,0.670264,0.999995,0.191667,0.999813,0.817211,0.402997,0.006107,...,0.500049,0.732364,0.006010,0.745906,0.601187,0.371982,1.000000,0.634294,0.948935,0.512086
C0198,0.641470,0.502546,0.579706,0.306603,0.634307,0.293963,0.634276,0.646828,0.948694,0.672265,...,0.395714,0.866233,0.004589,0.590309,0.569379,0.294048,0.634294,1.000000,0.502597,0.800119


5. Extract Top 3 Lookalikes

In [60]:
# Function to get top 3 similar customers for a given customer
def get_top_lookalikes(customer_id, similarity_df, top_n=3):
    similar_customers = similarity_df[customer_id].sort_values(ascending=False)[1:top_n+1]  # Exclude the customer itself
    return [(cust_id, score) for cust_id, score in similar_customers.items()]


In [61]:
# Create lookalike map for first 20 customers
lookalike_map = {}
for cust_id in similarity_df.index[:20]:  # First 20 customers
    lookalike_map[cust_id] = get_top_lookalikes(cust_id, similarity_df)



In [56]:
# Convert map to a DataFrame for export
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_map.keys(),
    'Lookalikes': [str(val) for val in lookalike_map.values()]  # Save as strings
})


In [57]:
lookalike_df

,CustomerID,Lookalikes
0,C0001,"[('C0146', 0.9849080916098838), ('C0035', 0.98..."
1,C0002,"[('C0133', 0.9997126131213345), ('C0144', 0.99..."
2,C0003,"[('C0166', 0.9999625836202699), ('C0031', 0.99..."
3,C0004,"[('C0085', 0.9844615400906513), ('C0047', 0.97..."
4,C0005,"[('C0197', 0.9999950820041779), ('C0007', 0.99..."
5,C0006,"[('C0135', 0.9995687626107228), ('C0147', 0.97..."
6,C0007,"[('C0005', 0.9998690212589688), ('C0197', 0.99..."
7,C0008,"[('C0181', 0.9816326409445046), ('C0154', 0.98..."
8,C0009,"[('C0040', 0.9958573802803893), ('C0092', 0.98..."
9,C0010,"[('C0077', 0.9899219209517904), ('C0083', 0.98..."


In [66]:
# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)